In [4]:
import sagemaker
import boto3
from sagemaker import get_execution_role

In [5]:
region = boto3.Session().region_name
sm_rt = boto3.Session().client('runtime.sagemaker', region_name=region)

ep_name = 'YearPredictionMSD-model'
id_name = ''
target = 'year'

file_name = "YearPredictionMSD_test.csv"
test_file_name = "YearPredictionMSD_test-no-label.csv"

kaggle_file_name = ""
kaggle_test_file_name = ""
kaggle_pred_file_name = ""

In [6]:
import pandas as pd
import numpy as np

df = pd.read_csv(file_name)
df = df.drop([target],axis=1) 
print(df.head())
print(df.shape)

df.to_csv(test_file_name, index=False)

         x0        x1        x2        x3        x4        x5       x6  \
0  53.57565  64.87821  51.57692  10.08200  -3.87556 -18.35914  7.27052   
1  53.88319  48.49841  45.70965   6.21980 -30.04902  -8.53448  6.71349   
2  49.75977  42.54686  -8.87435 -18.01974 -25.84566 -29.59054  3.48916   
3  49.46078  23.83484   6.78874  -8.87532  -3.29631 -19.14497 -8.55881   
4  51.21621  29.92536  26.34983  -3.67163 -48.88161 -19.69541 -9.14103   

         x7       x8       x9  ...       x80        x81       x82        x83  \
0   1.25718 -7.65948 -5.27055  ...  -6.69267  -34.61181  20.50827   16.83898   
1  -0.38959  2.12328  0.63870  ...  -1.73194  -50.43139  50.63848    6.05234   
2  -3.41514  6.30264  4.72971  ...  16.60815   86.10998  58.96147    1.24417   
3 -11.28947  2.94354  1.53887  ...  64.37474   42.57985  43.41533  147.92949   
4  -8.65851  2.16690  0.62977  ...   1.38520 -122.50344  12.10650   36.49427   

        x84       x85        x86      x87        x88       x89  
0  -2.330

In [7]:
f = open(test_file_name)
count = 0
pred = []

for line in f:
    if count==0:
        count=1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")
    pred.append(float(response))
    count = count + 1


In [8]:
print(pred[:20])

import pandas as pd
import numpy as np
pd = pd.read_csv(file_name)
y_true = np.array(pd[target])
y_pred = np.array(pred)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_true, y_pred)
print("test mse = {}".format(mse))

[1998.524658203125, 2002.6065673828125, 2004.369873046875, 2004.9896240234375, 1999.8597412109375, 2003.556640625, 1999.30517578125, 2003.9404296875, 1994.491943359375, 2003.3623046875, 2003.67822265625, 1990.9012451171875, 1997.35302734375, 1988.655029296875, 2002.707275390625, 2001.2862548828125, 1993.7510986328125, 2005.1400146484375, 2003.60107421875, 1979.8896484375]
test mse = 75.61080610368741


In [6]:
import pandas as pd
import numpy as np

df = pd.read_csv(kaggle_file_name)

kaggle_test_ids = np.array(df[id_name])

df = df.drop([id_name],axis=1) 
print(df.head())
print(df.shape)

df.to_csv(kaggle_test_file_name, index=False)

FileNotFoundError: [Errno 2] File  does not exist: ''

In [14]:
f = open(kaggle_test_file_name)
count = 0
kaggle_pred = []

for line in f:
    if count == 0:
        count = 1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")
    test_id = kaggle_test_ids[count-1]
    pred = int(response[0])
    kaggle_pred.append([test_id, pred])
    count = count + 1

kaggle_pred_df = pd.DataFrame(kaggle_pred, columns = [id_name, target])

kaggle_pred_df.to_csv(kaggle_pred_file_name, index=False)

In [5]:
print(kaggle_pred_df.head())
print(kaggle_pred_df.shape)

NameError: name 'kaggle_pred_df' is not defined

In [22]:
import boto3
sm = boto3.client('sagemaker')
#sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm.delete_endpoint(EndpointName=ep_name)

{'ResponseMetadata': {'RequestId': '3adbc464-12f4-4c5b-af4a-1bad8f737f35',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3adbc464-12f4-4c5b-af4a-1bad8f737f35',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 24 Mar 2021 07:51:25 GMT'},
  'RetryAttempts': 0}}